In [83]:
rm(list = ls())

In [84]:
# libraries
suppressMessages({
  library(tidyverse)
  library(broom)
  library(ranger)
}
)

In [85]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [86]:
dim(Data)

[1] 1460   81

In [87]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"  
 [5] "LotArea"       "Street"        "Alley"         "LotShape"     
 [9] "LandContour"   "Utilities"     "LotConfig"     "LandSlope"    
[13] "Neighborhood"  "Condition1"    "Condition2"    "BldgType"     
[17] "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"  
[25] "Exterior2nd"   "MasVnrType"    "MasVnrArea"    "ExterQual"    
[29] "ExterCond"     "Foundation"    "BsmtQual"      "BsmtCond"     
[33] "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"    "BsmtFinType2" 
[37] "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"    
[45] "X2ndFlrSF"     "LowQualFinSF"  "GrLivArea"     "BsmtFullBath" 
[49] "BsmtHalfBath"  "FullBath"      "HalfBath"      "BedroomAbvGr" 
[53] "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd"  "Functional"   
[57] "Fireplaces"    "FireplaceQu"

In [88]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [89]:
# numeric data
numData <- Data |> 
  select_if(is.numeric) 

In [90]:
hist(Data$SalePrice)

In [91]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [92]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [93]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [94]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [95]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

Let's impute some of these numerical features

In [96]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [97]:
# list of numeric features to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [98]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [99]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [100]:
# duplicates rows
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  nrow()

[1] 1460

In [101]:
# select numeric features
Data |> 
  select_if(is.numeric)  ->numData 

# check relationship with target (linear model)
# h0 : there no relationship between X and Y(SalePrice)
# h1 :otherwise

  lm(SalePrice~.,numData) |> 
    summary() |> 
    tidy() |> 
    filter(p.value <=0.05) |> 
    nrow()

[1] 19

28 numerical features have predictive effect on target feature. 

In [102]:
# correlation in numeric features
for (feature in names(numData)[-1]){
  if (cor(numData[names(numData)][1],numData[feature]) >= 0.8 ) print(feature)
}

There is no carrelated features in numeric features 

cat var feat engineering  

In [103]:
# categorical features presenting missing values 
sum(cat_skimData$n_missing  > 0)

[1] 16

In [104]:
# n_missing proportion > 0.95 in cat features 
cat_rid <- cat_skimData$skim_variable[cat_skimData$n_missing >= 0.95] 
length(cat_rid)

[1] 16

16 features present missing values or NA proportion above 0.95, we could get rid of it  

In [105]:
# remove these categorical features
Data[,cat_rid]<- NULL

In [106]:
# check categorical features with NA in Data 
sum(cat_rid %in% names(Data))
rm(cat_rid)

In [107]:
# new data dimension
dim(Data)

[1] 1460   64

In [108]:
# NA check
colMeans(is.na(Data)) |> sum()

[1] 0

There is no longer NA in this dataset

In [109]:
summary(lm(SalePrice~. ,Data)) |> 
  tidy() |> 
  filter(p.value <=0.05) |> 
  print(n=15)

# A tibble: 59 × 5
   term                  estimate std.error statistic  p.value
   <chr>                    <dbl>     <dbl>     <dbl>    <dbl>
 1 MSZoningFV           33495.    12123.         2.76 5.81e- 3
 2 MSZoningRH           25399.    12189.         2.08 3.74e- 2
 3 MSZoningRL           27888.    10386.         2.69 7.35e- 3
 4 MSZoningRM           26270.     9716.         2.70 6.95e- 3
 5 LotFrontage             96.2      44.0        2.19 2.89e- 2
 6 LotArea                  0.670     0.109      6.16 9.47e-10
 7 StreetPave           36874.    12184.         3.03 2.52e- 3
 8 LandContourHLS       13435.     5242.         2.56 1.05e- 2
 9 LandContourLvl        7446.     3775.         1.97 4.88e- 2
# A tibble: 59 × 5
   term                  estimate std.error statistic  p.value
   <chr>                    <dbl>     <dbl>     <dbl>    <dbl>
 1 MSZoningFV           33495.    12123.         2.76 5.81e- 3
 2 MSZoningRH           25399.    12189.         2.08 3.74e- 2
 3 MSZoningRL    

10 LotConfigCulDSac      9864.     3407.         2.90 3.86e- 3
11 LandSlopeMod         10064.     4002.         2.51 1.20e- 2
12 LandSlopeSev        -25446.    11049.        -2.30 2.14e- 2
13 NeighborhoodEdwards -18256.     8039.        -2.27 2.33e- 2
14 NeighborhoodMitchel -21908.     8233.        -2.66 7.89e- 3
15 NeighborhoodNAmes   -16956.     7887.        -2.15 3.18e- 2
# ℹ 44 more rows
# ℹ Use `print(n = ...)` to see more rows
10 LotConfigCulDSac      9864.     3407.         2.90 3.86e- 3
11 LandSlopeMod         10064.     4002.         2.51 1.20e- 2
12 LandSlopeSev        -25446.    11049.        -2.30 2.14e- 2
13 NeighborhoodEdwards -18256.     8039.        -2.27 2.33e- 2
14 NeighborhoodMitchel -21908.     8233.        -2.66 7.89e- 3
15 NeighborhoodNAmes   -16956.     7887.        -2.15 3.18e- 2
# ℹ 44 more rows
# ℹ Use `print(n = ...)` to see more rows


In [193]:
ranger(SalePrice~.,Data,
  importance = 'impurity') |> 
  importance() |> 
  sort(decreasing = T) |> 
  tidy() |> 
  mutate(proportion= x/sum(x)) |> 
  filter(proportion >= 0.06)

Warning message:
In tidy.numeric(sort(importance(ranger(SalePrice ~ ., Data, importance = "impurity")),  :
  'tidy.numeric' is deprecated.
See help("Deprecated")


# A tibble: 5 × 3
  names             x proportion
  <chr>         <dbl>      <dbl>
1 OverallQual 1.16e12     0.128 
2 GrLivArea   7.81e11     0.0867
3 GarageCars  6.22e11     0.0690
4 GarageArea  5.72e11     0.0634
5 TotalBsmtSF 5.71e11     0.0633

In [190]:
ranger(SalePrice~.,Data,importance = 'impurity')  -> rng
rng$variable.importance |> 
sort(decreasing = T) |> 
tidy() |> 
mutate(prop=x/sum(x)) |> 
#print(n=5) 
filter(prop>=0.05)

Warning message:
In tidy.numeric(sort(rng$variable.importance, decreasing = T)) :
  'tidy.numeric' is deprecated.
See help("Deprecated")


# A tibble: 7 × 3
  names             x   prop
  <chr>         <dbl>  <dbl>
1 OverallQual 1.09e12 0.122 
2 GrLivArea   8.30e11 0.0921
3 GarageCars  6.37e11 0.0707
4 TotalBsmtSF 5.97e11 0.0663
5 X1stFlrSF   5.17e11 0.0574
6 ExterQual   4.97e11 0.0551
7 GarageArea  4.79e11 0.0531

to be continued 😉!